In [1]:
%reload_ext autoreload
%autoreload 2

! [ ! -L /datasets ] && ln -s /data/datasets /datasets

from k12libs.utils.nb_easy import k12ai_set_notebook

k12ai_set_notebook(cellw=95)

## 需掌握知识点

1. 激活函数层(Relu, Sigmoid)
2. Dropout层
3. Batchnorm层

In [2]:
%reset -f

from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer
import torch
from torch import nn
from sklearn.metrics import confusion_matrix

class CustomClassifier(EasyaiClassifier):
    
    def __init__(self):
        super().__init__()
        self.input_size = 224 # 输入模型的图片大小
        self.batch_size = 64  # 输入模型的图片数量
        
        # 调试: 日志输出模型shape
        self.example_input_array = torch.zeros(
            self.batch_size, 3, self.input_size, self.input_size) 

    def prepare_dataset(self):
        """
        flowers: 22个分类
            0: 'salviasplendens' 鼠尾草
            1: 'daffodil' 水仙花
            2: 'snowdrop' 雪花莲
            3: 'lilyvalley' 铃兰花
            4: 'bluebell' 野风信子
            5: 'crocus' 番红花
            6: 'iris' 鸢尾花
            7: 'tigerlily' 卷丹
            8: 'tulip'  郁金香
            9: 'fritillary' 豹纹蝶
           10: 'sunflower' 向日葵
           11: 'daisy' 雏菊
           12: 'coltsfoot' 款冬
           13: 'dandelion' 蒲公英
           14: 'cowslip' 黄花九轮草
           15: 'buttercup' 毛茛
           16: 'windflower' 白头翁
           17: 'pansy' 蝴蝶花
           18: 'coxcomb'  鸡冠花
           19: 'flamingo' 红鹤
           20: 'lily'  百合花
           21: 'lotus' 荷花
        """
        return self.load_flowers()
    
    def build_model(self):
        class NeuralNet(nn.Module):
            def __init__(self, num_classes):
                super().__init__()
                self.features = nn.Sequential(
                    nn.BatchNorm2d(num_features=3, momentum=0.1, affine=True, track_running_stats=True),
                    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=1, dilation=1, groups=1, bias=True),
                    nn.ReLU(inplace=True),
                    nn.BatchNorm2d(num_features=64, momentum=0.1, affine=True, track_running_stats=True),
                    nn.MaxPool2d(kernel_size=7, stride=1, padding=3, ceil_mode=False),
                    nn.Dropout(inplace=True, p=0.5),
                    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=2, padding=2, dilation=1, groups=1, bias=True),
                    nn.ReLU(inplace=True),
                    nn.BatchNorm2d(num_features=64, momentum=0.1, affine=True, track_running_stats=True),
                    nn.MaxPool2d(kernel_size=5, stride=1, padding=2, ceil_mode=False),
                    nn.Dropout(inplace=True, p=0.5)
                )
                self.avgpool = nn.AdaptiveAvgPool2d(output_size=(5, 5))
                self.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(in_features=1600, out_features=128, bias=True),
                    nn.Dropout(inplace=True, p=0.5),
                    nn.Linear(in_features=128, out_features=num_classes, bias=True),
                  )
                
            def forward(self, x):     
                x = self.avgpool(self.features(x))
                return self.classifier(x)
            
        return NeuralNet(num_classes=22) # input shape: (32, 3, 224, 224)
    
    def train_dataloader(self):
        return self.get_dataloader('train', self.batch_size, self.input_size, normalize=True)

    def val_dataloader(self):
        return self.get_dataloader('val', self.batch_size, self.input_size, normalize=True)
    
    def test_dataloader(self):
        return self.get_dataloader('test', self.batch_size, self.input_size)
    
    def configure_optimizer(self, model):
        """
        优化器:
            adam: 利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率
            sgd:
        """
        # 优化算法
        # return self.adam(model.parameters(), base_lr=0.001, betas=(0.8, 0.88), weight_decay=0.001)
        return self.sgd(model.parameters(), base_lr=0.01, momentum=0.88, nesterov=False, weight_decay=0.001)
    
    def configure_scheduler(self, optimizer):
        """
        学习率策略:
            step_lr: 每step_size个epoch，lr会自动乘以gamma(lr = lr*gamma)
            multistep_lr: 按照milestones里的值,阶段地修改学习率(lr = lr*gamma)
            reduceon_lr: 监控val_loss的值, 如果在容忍patience值指定的epoch次数内没有减少, 则修改学习率(lr = lr*factor)
        """
        # return self.step_lr(optimizer, step_size=5, gamma=0.1)
        # return self.multistep_lr(optimizer, milestones=[5, 10], gamma=0.1)
        return self.reduceon_lr(optimizer, factor=0.1, patience=5, min_lr=0.000001)
    
    
trainer = EasyaiTrainer(
    max_epochs=2,
    log_lr=True,
    model_summary='full',
    early_stop={'monitor': 'val_loss', 'patience': 10, 'mode': 'min'},
    ckpt_path='9-1')

model = CustomClassifier()

# 训练
trainer.fit(model)

# 评估
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
{'label_names': ['salviasplendens',
                 'daffodil',
                 'snowdrop',
                 'lilyvalley',
                 'bluebell',
                 'crocus',
                 'iris',
                 'tigerlily',
                 'tulip',
                 'fritillary',
                 'sunflower',
                 'daisy',
                 'coltsfoot',
                 'dandelion',
                 'cowslip',
                 'buttercup',
                 'windflower',
                 'pansy',
                 'coxcomb',
                 'flamingo',
                 'lily',
                 'lotus'],
 'mean': [0.4623, 0.4305, 0.295],
 'num_classes': 22,
 'num_records': 1760,
 'std': [0.252, 0.2242, 0.2091]}

--------------------------------------------------------------------------------

   | Name               | Type              | Params | In sizes           | Out sizes         

Saving latest checkpoint..



(fit)	GPU-0 memory allocated: 0.0 MB	 max memory allocated: 1549.02 MB
--------------------------------------------------------------------------------
{'label_names': ['salviasplendens',
                 'daffodil',
                 'snowdrop',
                 'lilyvalley',
                 'bluebell',
                 'crocus',
                 'iris',
                 'tigerlily',
                 'tulip',
                 'fritillary',
                 'sunflower',
                 'daisy',
                 'coltsfoot',
                 'dandelion',
                 'cowslip',
                 'buttercup',
                 'windflower',
                 'pansy',
                 'coxcomb',
                 'flamingo',
                 'lily',
                 'lotus'],
 'mean': [0.4623, 0.4305, 0.295],
 'num_classes': 22,
 'num_records': 1760,
 'std': [0.252, 0.2242, 0.2091]}



(fit)	GPU-0 memory allocated: 1.23 MB	 max memory allocated: 1549.02 MB
(test)	GPU-0 memory allocated: 1.23 MB	 max memory allocated: 1549.02 MB
--------------------------------------------------------------------------------
{'test_acc': 0.09912367910146713}
--------------------------------------------------------------------------------
